# Hand Gesture Classification
### by Adrian Abraham

## 1 - Package installation
- **tensorflow**: for model creation and predictions
- **pandas**: for csv_reading
- **numpy**: for data storage and manipulation
- **mediapipe**: contains pre-made hand detection module for data collection
- **open-cv**: for live feed
- **ast**: to convert string literal lists into actual lists

In [1]:
!pip install tensorflow

In [2]:
!pip install pandas

In [3]:
!pip install mediapipe

In [4]:
!pip install opencv-python

## 2 - Importing packages

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.losses import BinaryCrossentropy

### Quick check that tensorflow is working

In [6]:
print(tf.__version__)

2.16.2


## 3 - Importing data

In [7]:
# Positive and negative data are stored in csv format.
# We can use read_csv() to get the contents of each file
df1 = pd.read_csv('peace.csv')
df0 = pd.read_csv('not_peace.csv')

In [8]:
df1.info

<bound method DataFrame.info of                                                  coords
0     [(0.6682692307692307, 1.0), (0.384615384615384...
1     [(0.6634615384615384, 1.0), (0.379807692307692...
2     [(0.6650717703349283, 1.0), (0.382775119617224...
3     [(0.6523809523809524, 1.0), (0.376190476190476...
4     [(0.6555023923444976, 1.0), (0.377990430622009...
...                                                 ...
1533  [(0.7243243243243244, 1.0), (0.491891891891891...
1534  [(0.7297297297297297, 1.0), (0.497297297297297...
1535  [(0.7297297297297297, 1.0), (0.491891891891891...
1536  [(0.7336956521739131, 1.0), (0.5, 0.9354838709...
1537  [(0.7336956521739131, 1.0), (0.5, 0.9354838709...

[1538 rows x 1 columns]>

In [9]:
df0.info

<bound method DataFrame.info of                                                  coords
0     [(0.75, 1.0), (0.5340909090909091, 0.946547884...
1     [(0.7154696132596685, 1.0), (0.511049723756906...
2     [(0.6941489361702128, 1.0), (0.486702127659574...
3     [(0.6851851851851852, 1.0), (0.473544973544973...
4     [(0.6487179487179487, 1.0), (0.446153846153846...
...                                                 ...
2007  [(0.7950530035335689, 1.0), (0.505300353356890...
2008  [(0.7758620689655172, 1.0), (0.496551724137931...
2009  [(0.75, 1.0), (0.48333333333333334, 0.93627450...
2010  [(0.7435897435897436, 1.0), (0.493589743589743...
2011  [(0.7272727272727273, 1.0), (0.482758620689655...

[2012 rows x 1 columns]>

In [10]:
# The format of our csv has a list of data as a string, 
# to turn them into lists so we use the ast library
df1['coords'] = df1['coords'].apply(ast.literal_eval)
positive_data = np.array(df1['coords'].tolist())

df0['coords'] = df0['coords'].apply(ast.literal_eval)
negative_data = np.array(df0['coords'].tolist())

In [11]:
positive_data

array([[[0.66826923, 1.        ],
        [0.38461538, 0.93271462],
        [0.24038462, 0.78190255],
        ...,
        [0.97115385, 0.59860789],
        [0.85096154, 0.67981439],
        [0.77403846, 0.75638051]],

       [[0.66346154, 1.        ],
        [0.37980769, 0.93518519],
        [0.24038462, 0.78703704],
        ...,
        [0.97115385, 0.59953704],
        [0.85096154, 0.68287037],
        [0.77403846, 0.75925926]],

       [[0.66507177, 1.        ],
        [0.38277512, 0.93317972],
        [0.23923445, 0.78571429],
        ...,
        [0.98086124, 0.60368664],
        [0.86124402, 0.68663594],
        [0.77511962, 0.76267281]],

       ...,

       [[0.72972973, 1.        ],
        [0.49189189, 0.93261456],
        [0.37837838, 0.78436658],
        ...,
        [0.95135135, 0.66846361],
        [0.83243243, 0.74663073],
        [0.76756757, 0.80862534]],

       [[0.73369565, 1.        ],
        [0.5       , 0.93548387],
        [0.38043478, 0.78763441],
        .

In [12]:
negative_data

array([[[0.75      , 1.        ],
        [0.53409091, 0.94654788],
        [0.36079545, 0.82628062],
        ...,
        [0.95738636, 0.47216036],
        [0.98295455, 0.38975501],
        [1.        , 0.30957684]],

       [[0.71546961, 1.        ],
        [0.51104972, 0.94130435],
        [0.34530387, 0.8173913 ],
        ...,
        [0.95303867, 0.4826087 ],
        [0.98066298, 0.4       ],
        [1.        , 0.3173913 ]],

       [[0.69414894, 1.        ],
        [0.48670213, 0.93418259],
        [0.32446809, 0.80679406],
        ...,
        [0.94414894, 0.48619958],
        [0.97606383, 0.40127389],
        [1.        , 0.32059448]],

       ...,

       [[0.75      , 1.        ],
        [0.48333333, 0.93627451],
        [0.31333333, 0.77941176],
        ...,
        [0.98      , 0.46078431],
        [0.99333333, 0.37990196],
        [1.        , 0.29901961]],

       [[0.74358974, 1.        ],
        [0.49358974, 0.93364929],
        [0.32051282, 0.78199052],
        .

In [13]:
# We can concatenate them together to have all our data in one np array
data = np.concatenate((positive_data,negative_data))
data.shape

(3550, 21, 2)

In [14]:
# Each training example can be either 1 or 0
targets = np.zeros(data.shape[0])

# We are setting each training example to have either 0 or 1 output
for i in range(positive_data.shape[0]):
    targets[i] = 1

In [15]:
model = Sequential([
    Input(shape=(21,2)),          # Define the input shape here
    Flatten(),                     # Flatten the (21, 2) input into a 1D vector
    Dense(25, activation='relu'), # Fully connected layer with 128 neurons
    Dense(10, activation='relu'),  # Fully connected layer with 64 neurons
    Dense(1, activation='sigmoid') # Output layer for binary classification
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,075 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,346 (5.26 KB)

 Trainable params: 1,346 (5.26 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),  # Use appropriate loss function for classification
              metrics=['accuracy'])

In [17]:
history = model.fit(
    data,targets,
    epochs=40
)

Epoch 1/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 530us/step - accuracy: 0.7025 - loss: 0.6444
Epoch 2/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.9842 - loss: 0.3205
Epoch 3/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9945 - loss: 0.0916
Epoch 4/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - accuracy: 1.0000 - loss: 0.0373
Epoch 5/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 1.0000 - loss: 0.0188
Epoch 6/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step - accuracy: 1.0000 - loss: 0.0120
Epoch 7/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 1.0000 - loss: 0.0076
Epoch 8/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 1.0000 - loss: 0.0055
Epoch 9/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 1.0000 - loss: 0.0042
Epoch 10/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - accuracy: 1.0000 - loss: 0.0035
Epoch 11/40
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - accuracy: 1.0000 - loss: 0.0027
Epoch 12/40
111/111 ━━━━━━━━━━━━

In [19]:
import mediapipe as mp
import cv2

In [20]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1720224939.244129 15070763 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1720224939.268491 15073106 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720224939.274407 15073106 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [21]:
cap = cv2.VideoCapture(0)
BOX_MARGIN = 50
while cap.isOpened():
    # capturing the current frame
    ret, frame = cap.read()
    # getting window dimensions, shape contains height, width, and channels
    height, width, _ = frame.shape

    # if no frame returned, break
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        # for each hand found we calculate the coords of the landmarks
        for hand_landmarks in results.multi_hand_landmarks:
            hand_coords = np.array([ (int(landmark.x * width), int(landmark.y * height)) for landmark in hand_landmarks.landmark ])
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        hand_x_min, hand_x_max = min(x for x,y in hand_coords), max(x for x,y in hand_coords)
        hand_y_min, hand_y_max = min(y for x,y in hand_coords), max(y for x,y in hand_coords)
        
        rect_start, rect_end = (hand_x_min - BOX_MARGIN, hand_y_min - BOX_MARGIN), (hand_x_max + BOX_MARGIN, hand_y_max + BOX_MARGIN)
        rel_coords = np.array([ (x - rect_start[0], y - rect_start[1]) for x,y in hand_coords ])
        
        rel_x_max = max(x for x,y in rel_coords)
        rel_y_max = max(y for x,y in rel_coords)

        normalized_rel_coords = np.array([ (x / rel_x_max, y / rel_y_max) for x,y in rel_coords ])
        normalized_rel_coords = np.expand_dims(normalized_rel_coords, axis=0)

        prediction = model.predict(normalized_rel_coords)[0][0]
        if prediction > 0.9:
            text = f"peace be with you, Confidence: {round(prediction * 100,2)}%"
        else:
            text = f"give me some peace bro, Confidence: {round(prediction * 100,2)}%"

        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
    cv2.imshow('Hand Landmarks', frame)

    # when q pressed, we end video capture
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        


[07/05 17:15:40.255893][info][15070763][Context.cpp:69] Context created with config: default config!
[07/05 17:15:40.255976][info][15070763][Context.cpp:74] Context work_dir=/Users/adrianpaul1/Desktop/jupyter/hand-gesture-recognition/pythonProject1
[07/05 17:15:40.255980][info][15070763][Context.cpp:77] 	- SDK version: 1.9.4
[07/05 17:15:40.255982][info][15070763][Context.cpp:78] 	- SDK stage version: main
[07/05 17:15:40.255988][info][15070763][Context.cpp:82] get config EnumerateNetDevice:false
[07/05 17:15:40.255992][info][15070763][MacPal.cpp:36] createObPal: create MacPal!
[07/05 17:15:40.257760][info][15070763][MacPal.cpp:104] Create PollingDeviceWatcher!
[07/05 17:15:40.257792][info][15070763][DeviceManager.cpp:15] Current found device(s): (0)
[07/05 17:15:40.257798][info][15070763][Pipeline.cpp:15] Try to create pipeline with default device.
[07/05 17:15:40.257835][warning][15070763][ObException.cpp:5] No device found, fail to create pipeline!
[07/05 17:15:40.260273][info][1507

OpenCV: not authorized to capture video (status 0), requesting...
OpenCV: camera failed to properly initialize!
[ WARN:0@3.347] global cap.cpp:323 open VIDEOIO(OBSENSOR): raised unknown C++ exception!




In [ ]:
type(prediction[0])